In [3]:
%load_ext autoreload
%autoreload

from os import getcwd
from os.path import join, abspath, pardir, relpath, exists

import requests

from dataclasses import dataclass, field

import pandas as pd
import numpy as np
from numpy import matrixlib as npmat
import networkx as nx
from typing import Union
import pulp as p
from itertools import combinations
from typing import List, Tuple
from enum import Enum
from scipy.stats import kendalltau, spearmanr

from IPython.display import IFrame
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
pd.set_option('display.max_columns', None)
parent_dir = abspath(join(getcwd(), pardir))
data_dir = join(parent_dir, 'data')
data_file = join(data_dir, 'products.csv')
parent_dir

'/Users/mohammadzainabbas/Masters/CS/Decision-Modelling/nutri-score'

In [5]:
df = pd.read_csv(data_file)
df.head()

,id,name,image_url,brand,categories,energy,energy_points,energy_value,fiber,fiber_points,fiber_value,fruits_vegetables_nuts_colza_walnut_olive_oils,fruits_vegetables_nuts_colza_walnut_olive_oils_points,fruits_vegetables_nuts_colza_walnut_olive_oils_value,grade,is_beverage,is_cheese,is_fat,is_water,negative_points,positive_points,proteins,proteins_points,proteins_value,saturated_fat,saturated_fat_points,saturated_fat_ratio,saturated_fat_ratio_points,saturated_fat_ratio_value,saturated_fat_value,score,sodium,sodium_points,sodium_value,sugars,sugars_points,sugars_value,calcium,calcium_100g,calcium_label,calcium_unit,calcium_value,carbohydrates,carbohydrates_100g,carbohydrates_unit,carbohydrates_value,energy-kcal,energy-kcal_100g,energy-kcal_unit,energy-kcal_value,energy-kj,energy-kj_100g,energy-kj_unit,energy-kj_value,energy_100g,energy_unit,fat,fat_100g,fat_unit,fat_value,fiber_100g,fiber_unit,fruits-vegetables-nuts-estimate-from-ingredients_100g,fruits-vegetables-nuts-estimate-from-ingredients_serving,nova-group,nova-group_100g,nova-group_serving,nutrition-score-fr,nutrition-score-fr_100g,proteins_100g,proteins_unit,salt,salt_100g,salt_unit,salt_value,saturated-fat,saturated-fat_100g,saturated-fat_unit,saturated-fat_value,sodium_100g,sodium_unit,sugars_100g,sugars_unit,vitamin-b1,vitamin-b1_100g,vitamin-b1_unit,vitamin-b1_value,vitamin-b9,vitamin-b9_100g,vitamin-b9_unit,vitamin-b9_value,vitamin-b12,vitamin-b12_100g,vitamin-b12_unit,vitamin-b12_value,vitamin-d,vitamin-d_100g,vitamin-d_unit,vitamin-d_value,carbohydrates_serving,energy-kcal_serving,energy-kcal_value_computed,energy-kj_serving,energy-kj_value_computed,energy_serving,fat_serving,fiber_serving,fruits-vegetables-nuts-estimate,fruits-vegetables-nuts-estimate_100g,fruits-vegetables-nuts-estimate_label,fruits-vegetables-nuts-estimate_serving,fruits-vegetables-nuts-estimate_unit,fruits-vegetables-nuts-estimate_value,proteins_serving,salt_serving,saturated-fat_serving,sodium_serving,sugars_serving,carbon-footprint-from-known-ingredients_100g,carbon-footprint-from-known-ingredients_product,carbon-footprint-from-known-ingredients_serving,alcohol,alcohol_100g,alcohol_serving,alcohol_unit,alcohol_value,fat_modifier,saturated-fat_modifier,fruits-vegetables-nuts,fruits-vegetables-nuts_100g,fruits-vegetables-nuts_label,fruits-vegetables-nuts_serving,fruits-vegetables-nuts_unit,fruits-vegetables-nuts_value,calcium_serving,cholesterol,cholesterol_100g,cholesterol_serving,cholesterol_unit,cholesterol_value,folates,folates_100g,folates_serving,folates_unit,folates_value,iron,iron_100g,iron_serving,iron_unit,iron_value,monounsaturated-fat,monounsaturated-fat_100g,monounsaturated-fat_serving,monounsaturated-fat_unit,monounsaturated-fat_value,polyunsaturated-fat,polyunsaturated-fat_100g,polyunsaturated-fat_serving,polyunsaturated-fat_unit,polyunsaturated-fat_value,trans-fat,trans-fat_100g,trans-fat_serving,trans-fat_unit,trans-fat_value,vitamin-a,vitamin-a_100g,vitamin-a_serving,vitamin-a_unit,vitamin-a_value,vitamin-b1_serving,vitamin-b2,vitamin-b2_100g,vitamin-b2_serving,vitamin-b2_unit,vitamin-b2_value,vitamin-c,vitamin-c_100g,vitamin-c_serving,vitamin-c_unit,vitamin-c_value,vitamin-pp,vitamin-pp_100g,vitamin-pp_serving,vitamin-pp_unit,vitamin-pp_value,polyols,polyols_100g,polyols_label,polyols_serving,polyols_unit,polyols_value,sugars_modifier,alpha-linolenic-acid,alpha-linolenic-acid_100g,alpha-linolenic-acid_unit,alpha-linolenic-acid_value,vitamin-e,vitamin-e_100g,vitamin-e_unit,vitamin-e_value,salt_modifier,sodium_modifier,lactose,lactose_100g,lactose_label,lactose_unit,lactose_value,vitamin-b12_label,vitamin-b2_label,vitamin-d_label,vitamin-e_label,carbohydrates_prepared,carbohydrates_prepared_100g,carbohydrates_prepared_serving,carbohydrates_prepared_unit,carbohydrates_prepared_value,energy-kcal_prepared,energy-kcal_prepared_100g,energy-kcal_prepared_serving,energy-kcal_prepared_unit,energy-kcal_prepared_value,energy-kj_prepared,energy-kj_prepared_100g,energy-kj_prepared_servin

In [6]:
df.columns.to_list()

['id',
 'name',
 'image_url',
 'brand',
 'categories',
 'energy',
 'energy_points',
 'energy_value',
 'fiber',
 'fiber_points',
 'fiber_value',
 'fruits_vegetables_nuts_colza_walnut_olive_oils',
 'fruits_vegetables_nuts_colza_walnut_olive_oils_points',
 'fruits_vegetables_nuts_colza_walnut_olive_oils_value',
 'grade',
 'is_beverage',
 'is_cheese',
 'is_fat',
 'is_water',
 'negative_points',
 'positive_points',
 'proteins',
 'proteins_points',
 'proteins_value',
 'saturated_fat',
 'saturated_fat_points',
 'saturated_fat_ratio',
 'saturated_fat_ratio_points',
 'saturated_fat_ratio_value',
 'saturated_fat_value',
 'score',
 'sodium',
 'sodium_points',
 'sodium_value',
 'sugars',
 'sugars_points',
 'sugars_value',
 'calcium',
 'calcium_100g',
 'calcium_label',
 'calcium_unit',
 'calcium_value',
 'carbohydrates',
 'carbohydrates_100g',
 'carbohydrates_unit',
 'carbohydrates_value',
 'energy-kcal',
 'energy-kcal_100g',
 'energy-kcal_unit',
 'energy-kcal_value',
 'energy-kj',
 'energy-kj_100g

In [7]:
df[df["negative_points"] != (df["energy_points"] + df["saturated_fat_points"] + df["sugars_points"] + df["sodium_points"])]

,id,name,image_url,brand,categories,energy,energy_points,energy_value,fiber,fiber_points,fiber_value,fruits_vegetables_nuts_colza_walnut_olive_oils,fruits_vegetables_nuts_colza_walnut_olive_oils_points,fruits_vegetables_nuts_colza_walnut_olive_oils_value,grade,is_beverage,is_cheese,is_fat,is_water,negative_points,positive_points,proteins,proteins_points,proteins_value,saturated_fat,saturated_fat_points,saturated_fat_ratio,saturated_fat_ratio_points,saturated_fat_ratio_value,saturated_fat_value,score,sodium,sodium_points,sodium_value,sugars,sugars_points,sugars_value,calcium,calcium_100g,calcium_label,calcium_unit,calcium_value,carbohydrates,carbohydrates_100g,carbohydrates_unit,carbohydrates_value,energy-kcal,energy-kcal_100g,energy-kcal_unit,energy-kcal_value,energy-kj,energy-kj_100g,energy-kj_unit,energy-kj_value,energy_100g,energy_unit,fat,fat_100g,fat_unit,fat_value,fiber_100g,fiber_unit,fruits-vegetables-nuts-estimate-from-ingredients_100g,fruits-vegetables-nuts-estimate-from-ingredients_serving,nova-group,nova-group_100g,nova-group_serving,nutrition-score-fr,nutrition-score-fr_100g,proteins_100g,proteins_unit,salt,salt_100g,salt_unit,salt_value,saturated-fat,saturated-fat_100g,saturated-fat_unit,saturated-fat_value,sodium_100g,sodium_unit,sugars_100g,sugars_unit,vitamin-b1,vitamin-b1_100g,vitamin-b1_unit,vitamin-b1_value,vitamin-b9,vitamin-b9_100g,vitamin-b9_unit,vitamin-b9_value,vitamin-b12,vitamin-b12_100g,vitamin-b12_unit,vitamin-b12_value,vitamin-d,vitamin-d_100g,vitamin-d_unit,vitamin-d_value,carbohydrates_serving,energy-kcal_serving,energy-kcal_value_computed,energy-kj_serving,energy-kj_value_computed,energy_serving,fat_serving,fiber_serving,fruits-vegetables-nuts-estimate,fruits-vegetables-nuts-estimate_100g,fruits-vegetables-nuts-estimate_label,fruits-vegetables-nuts-estimate_serving,fruits-vegetables-nuts-estimate_unit,fruits-vegetables-nuts-estimate_value,proteins_serving,salt_serving,saturated-fat_serving,sodium_serving,sugars_serving,carbon-footprint-from-known-ingredients_100g,carbon-footprint-from-known-ingredients_product,carbon-footprint-from-known-ingredients_serving,alcohol,alcohol_100g,alcohol_serving,alcohol_unit,alcohol_value,fat_modifier,saturated-fat_modifier,fruits-vegetables-nuts,fruits-vegetables-nuts_100g,fruits-vegetables-nuts_label,fruits-vegetables-nuts_serving,fruits-vegetables-nuts_unit,fruits-vegetables-nuts_value,calcium_serving,cholesterol,cholesterol_100g,cholesterol_serving,cholesterol_unit,cholesterol_value,folates,folates_100g,folates_serving,folates_unit,folates_value,iron,iron_100g,iron_serving,iron_unit,iron_value,monounsaturated-fat,monounsaturated-fat_100g,monounsaturated-fat_serving,monounsaturated-fat_unit,monounsaturated-fat_value,polyunsaturated-fat,polyunsaturated-fat_100g,polyunsaturated-fat_serving,polyunsaturated-fat_unit,polyunsaturated-fat_value,trans-fat,trans-fat_100g,trans-fat_serving,trans-fat_unit,trans-fat_value,vitamin-a,vitamin-a_100g,vitamin-a_serving,vitamin-a_unit,vitamin-a_value,vitamin-b1_serving,vitamin-b2,vitamin-b2_100g,vitamin-b2_serving,vitamin-b2_unit,vitamin-b2_value,vitamin-c,vitamin-c_100g,vitamin-c_serving,vitamin-c_unit,vitamin-c_value,vitamin-pp,vitamin-pp_100g,vitamin-pp_serving,vitamin-pp_unit,vitamin-pp_value,polyols,polyols_100g,polyols_label,polyols_serving,polyols_unit,polyols_value,sugars_modifier,alpha-linolenic-acid,alpha-linolenic-acid_100g,alpha-linolenic-acid_unit,alpha-linolenic-acid_value,vitamin-e,vitamin-e_100g,vitamin-e_unit,vitamin-e_value,salt_modifier,sodium_modifier,lactose,lactose_100g,lactose_label,lactose_unit,lactose_value,vitamin-b12_label,vitamin-b2_label,vitamin-d_label,vitamin-e_label,carbohydrates_prepared,carbohydrates_prepared_100g,carbohydrates_prepared_serving,carbohydrates_prepared_unit,carbohydrates_prepared_value,energy-kcal_prepared,energy-kcal_prepared_100g,energy-kcal_prepared_serving,energy-kcal_prepared_unit,energy-kcal_prepared_value,energy-kj_prepared,energy-kj_prepared_100g,energy-kj_prepared_servin

In [8]:
df[df["id"] == 8480024002679]

,id,name,image_url,brand,categories,energy,energy_points,energy_value,fiber,fiber_points,fiber_value,fruits_vegetables_nuts_colza_walnut_olive_oils,fruits_vegetables_nuts_colza_walnut_olive_oils_points,fruits_vegetables_nuts_colza_walnut_olive_oils_value,grade,is_beverage,is_cheese,is_fat,is_water,negative_points,positive_points,proteins,proteins_points,proteins_value,saturated_fat,saturated_fat_points,saturated_fat_ratio,saturated_fat_ratio_points,saturated_fat_ratio_value,saturated_fat_value,score,sodium,sodium_points,sodium_value,sugars,sugars_points,sugars_value,calcium,calcium_100g,calcium_label,calcium_unit,calcium_value,carbohydrates,carbohydrates_100g,carbohydrates_unit,carbohydrates_value,energy-kcal,energy-kcal_100g,energy-kcal_unit,energy-kcal_value,energy-kj,energy-kj_100g,energy-kj_unit,energy-kj_value,energy_100g,energy_unit,fat,fat_100g,fat_unit,fat_value,fiber_100g,fiber_unit,fruits-vegetables-nuts-estimate-from-ingredients_100g,fruits-vegetables-nuts-estimate-from-ingredients_serving,nova-group,nova-group_100g,nova-group_serving,nutrition-score-fr,nutrition-score-fr_100g,proteins_100g,proteins_unit,salt,salt_100g,salt_unit,salt_value,saturated-fat,saturated-fat_100g,saturated-fat_unit,saturated-fat_value,sodium_100g,sodium_unit,sugars_100g,sugars_unit,vitamin-b1,vitamin-b1_100g,vitamin-b1_unit,vitamin-b1_value,vitamin-b9,vitamin-b9_100g,vitamin-b9_unit,vitamin-b9_value,vitamin-b12,vitamin-b12_100g,vitamin-b12_unit,vitamin-b12_value,vitamin-d,vitamin-d_100g,vitamin-d_unit,vitamin-d_value,carbohydrates_serving,energy-kcal_serving,energy-kcal_value_computed,energy-kj_serving,energy-kj_value_computed,energy_serving,fat_serving,fiber_serving,fruits-vegetables-nuts-estimate,fruits-vegetables-nuts-estimate_100g,fruits-vegetables-nuts-estimate_label,fruits-vegetables-nuts-estimate_serving,fruits-vegetables-nuts-estimate_unit,fruits-vegetables-nuts-estimate_value,proteins_serving,salt_serving,saturated-fat_serving,sodium_serving,sugars_serving,carbon-footprint-from-known-ingredients_100g,carbon-footprint-from-known-ingredients_product,carbon-footprint-from-known-ingredients_serving,alcohol,alcohol_100g,alcohol_serving,alcohol_unit,alcohol_value,fat_modifier,saturated-fat_modifier,fruits-vegetables-nuts,fruits-vegetables-nuts_100g,fruits-vegetables-nuts_label,fruits-vegetables-nuts_serving,fruits-vegetables-nuts_unit,fruits-vegetables-nuts_value,calcium_serving,cholesterol,cholesterol_100g,cholesterol_serving,cholesterol_unit,cholesterol_value,folates,folates_100g,folates_serving,folates_unit,folates_value,iron,iron_100g,iron_serving,iron_unit,iron_value,monounsaturated-fat,monounsaturated-fat_100g,monounsaturated-fat_serving,monounsaturated-fat_unit,monounsaturated-fat_value,polyunsaturated-fat,polyunsaturated-fat_100g,polyunsaturated-fat_serving,polyunsaturated-fat_unit,polyunsaturated-fat_value,trans-fat,trans-fat_100g,trans-fat_serving,trans-fat_unit,trans-fat_value,vitamin-a,vitamin-a_100g,vitamin-a_serving,vitamin-a_unit,vitamin-a_value,vitamin-b1_serving,vitamin-b2,vitamin-b2_100g,vitamin-b2_serving,vitamin-b2_unit,vitamin-b2_value,vitamin-c,vitamin-c_100g,vitamin-c_serving,vitamin-c_unit,vitamin-c_value,vitamin-pp,vitamin-pp_100g,vitamin-pp_serving,vitamin-pp_unit,vitamin-pp_value,polyols,polyols_100g,polyols_label,polyols_serving,polyols_unit,polyols_value,sugars_modifier,alpha-linolenic-acid,alpha-linolenic-acid_100g,alpha-linolenic-acid_unit,alpha-linolenic-acid_value,vitamin-e,vitamin-e_100g,vitamin-e_unit,vitamin-e_value,salt_modifier,sodium_modifier,lactose,lactose_100g,lactose_label,lactose_unit,lactose_value,vitamin-b12_label,vitamin-b2_label,vitamin-d_label,vitamin-e_label,carbohydrates_prepared,carbohydrates_prepared_100g,carbohydrates_prepared_serving,carbohydrates_prepared_unit,carbohydrates_prepared_value,energy-kcal_prepared,energy-kcal_prepared_100g,energy-kcal_prepared_serving,energy-kcal_prepared_unit,energy-kcal_prepared_value,energy-kj_prepared,energy-kj_prepared_100g,energy-kj_prepared_servin